# Library

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi

Tue Jun  7 18:33:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
!pip install import-ipynb
!pip install albumentations==0.4.6
#!pip install -U efficientnet==0.0.4
#!git clone https://github.com/qubvel/efficientnet.git
#!pip install -U efficientnet
!pip install -U --pre efficientnet
!pip install keras-unet-collection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 117 kB 32.3 MB/s 
     |████████████████████████████████| 948 kB 75.4 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=cab4eeca2fe2856f228ef9393329f7771c592d1010d79ac5cce80a94ca760b45
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
Looking in indexes: https://pypi.org/sim

In [6]:
import sys
import os
import glob
from PIL import Image
from IPython.display import clear_output
import math
import random
import scipy.io as sio
import re
import time
from tensorflow.keras import models,layers
from tensorflow.keras.utils import get_file
from collections import namedtuple
from tqdm import tqdm, tqdm_notebook
from fastprogress.fastprogress import master_bar, progress_bar
from time import sleep

import numpy as np
import pandas as pd

import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid import ImageGrid
%matplotlib inline
import seaborn as sns
import zipfile
from skimage import io

import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.metrics import *
import tensorflow.keras.backend as K
from sklearn.preprocessing import StandardScaler, normalize
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from tensorflow.keras.utils import Progbar
from tensorflow.keras.layers.experimental import preprocessing
from IPython.display import display
from efficientnet.keras import *
from keras_unet_collection import models

%cd /content/drive/MyDrive/brain_mri_segmentation/PRANet-Polyps-Segmentation/
from model.ra_module import ReverseAttention
from model.partial_decoder import PartialDecoder
from model.rfb import RFB
from model.backbone import FE_backbone

%cd /content/drive/MyDrive/MRI_ACDC/
from keras_vision_transformer import swin_layers
from keras_vision_transformer import transformer_layers
from keras_vision_transformer import utils

seed = 1234
random.seed = seed
np.random.seed = seed
smooth = 1e-15
epochs = 300

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.


/content/drive/.shortcut-targets-by-id/1Dojj8OTWQKrTtjnAqGIo975J73GnTnMb/brain_mri_segmentation/PRANet-Polyps-Segmentation
/content/drive/.shortcut-targets-by-id/1JR20ypbQtsjiSTxgEkWdBHUTCTNJfWGR/MRI_ACDC


# Pre-Processing

In [7]:
# Data Path
TRAIN_PATH = '/content/drive/MyDrive/brain_mri_segmentation/dataset/cell_segmentation/'
#TEST_PATH = 'stage1_test/'

# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
#test_ids = next(os.walk(TEST_PATH))[1]

In [8]:
def normalize(image,mask):
  image = tf.cast(image, tf.float32) / 255.0
  mask = tf.cast(mask, tf.float32)
  return image,mask

In [9]:
# Function read train images and mask return as nump array
def read_train_data(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3):
    X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    print('Getting and resizing train images and masks ... ')
    sys.stdout.flush()
    if os.path.isfile("/content/drive/MyDrive/brain_mri_segmentation/dataset/x_train_skin_2018.npy") and os.path.isfile("/content/drive/MyDrive/brain_mri_segmentation/dataset/y_train_skin_2018.npy"):
        print("Train file loaded from memory")
        X_train = np.load("/content/drive/MyDrive/brain_mri_segmentation/dataset/x_train_skin_2018.npy")
        Y_train = np.load("/content/drive/MyDrive/brain_mri_segmentation/dataset/y_train_skin_2018.npy")
        return X_train,Y_train
    a = Progbar(len(train_ids))
    for n, id_ in enumerate(train_ids):
        path = TRAIN_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_train[n] = img
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = imread(path + '/masks/' + mask_file)
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                        preserve_range=True), axis=-1)
            mask = np.maximum(mask, mask_)
        Y_train[n] = mask
        a.update(n)
    #np.save("/content/drive/MyDrive/brain_mri_segmntateion/dataset/x_train_cell.npy",X_train)
    #np.save("/content/drive/MyDrive/brain_mri_segmntateion/dataset/y_train_cell.npy",Y_train)
    return X_train,Y_train

# get train_data
X_train, Y_train = read_train_data()

# get test_data
#test_img,test_img_sizes = read_test_data()

Getting and resizing train images and masks ... 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


Train file loaded from memory


In [10]:
#np.save("/content/drive/MyDrive/brain_mri_segmentation/dataset/x_train_cell.npy", x_train)
#np.save("/content/drive/MyDrive/brain_mri_segmentation/dataset/y_train_cell.npy", y_train)

In [12]:
x_train, y_train = normalize(X_train, Y_train)

len_dev = X_train.shape[0]//5
len_train = X_train.shape[0] - len_dev
rand_index = np.random.permutation(X_train.shape[0])
x_test = X_train[:len_dev]
y_test = Y_train[:len_dev]
x_train = X_train[len_dev:]
y_train = Y_train[len_dev:]

# Segmentation Training

## Preparation

In [ ]:
def mvn(tensor):
    '''Performs per-channel spatial mean-variance normalization.'''
    epsilon = 1e-6
    mean = K.mean(tensor, axis=(1,2), keepdims=True)
    std = K.std(tensor, axis=(1,2), keepdims=True)
    mvn = (tensor - mean) / (std + epsilon)
    
    return mvn

In [ ]:
def crop(tensors):
    '''
    List of 2 tensors, the second tensor having larger spatial dimensions.
    '''
    h_dims, w_dims = [], []
    for t in tensors:
        b, h, w, d = K.int_shape(t)
        h_dims.append(h)
        w_dims.append(w)
    crop_h, crop_w = (h_dims[1] - h_dims[0]), (w_dims[1] - w_dims[0])
    rem_h = int(crop_h % 2)
    rem_w = int(crop_w % 2)
    tt_h = int(crop_h / 2)
    tt_w = int(crop_w / 2)
    crop_h_dims = (tt_h, tt_h + rem_h)
    crop_w_dims = (tt_w, tt_w + rem_w)
    cropped = Cropping2D(cropping=(crop_h_dims, crop_w_dims))(tensors[1])
    
    return cropped

In [ ]:
def bn_act(x, act=True):
    x = tf.keras.layers.BatchNormalization()(x)
    if act == True:
        x = tf.keras.layers.Activation("relu")(x)
    return x 

def _conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = bn_act(x)
    conv = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv 

def stem(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = _conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)

    shortcut = tf.keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)

    output = tf.keras.layers.Add()([conv, shortcut])
    return output 

def _residual_block(x, filters, kernel_size=(3, 3), padding='same', strides=1):
    res = _conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    res = _conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)

    shortcut = tf.keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = tf.keras.layers.Add()([shortcut, res])
    return output

def upsample_concat_block(x, xskip):
    u = tf.keras.layers.UpSampling2D((2, 2))(x)
    c = tf.keras.layers.Concatenate()([u, xskip])
    return c 

In [ ]:
def up_and_concate(down_layer, layer, data_format='channels_last'):
    data_format='channels_last'
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))

    concate = my_concat([up, layer])

    return concate
def attention_up_and_concate(down_layer, layer, data_format='channels_last'):
    data_format='channels_last'
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)

    layer = attention_block_2d(x=layer, g=up, inter_channel=in_channel // 4, data_format=data_format)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))

    concate = my_concat([up, layer])
    return concate
def attention_block_2d(x, g, inter_channel, data_format='channels_last'):
    data_format='channels_last'
    # theta_x(?,g_height,g_width,inter_channel)

    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(x)

    # phi_g(?,g_height,g_width,inter_channel)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(g)

    # f(?,g_height,g_width,inter_channel)

    f = Activation('relu')(add([theta_x, phi_g]))

    # psi_f(?,g_height,g_width,1)

    psi_f = Conv2D(1, [1, 1], strides=[1, 1], data_format=data_format)(f)

    rate = Activation('sigmoid')(psi_f)

    # rate(?,x_height,x_width)

    # att_x(?,x_height,x_width,x_channel)

    att_x = multiply([x, rate])

    return att_x
def res_block(input_layer, out_n_filters, batch_normalization=False, kernel_size=[3, 3], stride=[1, 1],

              padding='same', data_format='channels_first'):
    data_format='channels_last'
    if data_format == 'channels_first':
        input_n_filters = input_layer.get_shape().as_list()[1]
    else:
        input_n_filters = input_layer.get_shape().as_list()[3]

    layer = input_layer
    for i in range(2):
        layer = Conv2D(out_n_filters // 4, [1, 1], strides=stride, padding=padding, data_format=data_format)(layer)
        if batch_normalization:
            layer = BatchNormalization()(layer)
        layer = Activation('relu')(layer)
        layer = Conv2D(out_n_filters // 4, kernel_size, strides=stride, padding=padding, data_format=data_format)(layer)
        layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(layer)

    if out_n_filters != input_n_filters:
        skip_layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(
            input_layer)
    else:
        skip_layer = input_layer
    out_layer = add([layer, skip_layer])
    return out_layer


# Recurrent Residual Convolutional Neural Network based on U-Net (R2U-Net)
def rec_res_block(input_layer, out_n_filters, batch_normalization=False, kernel_size=[3, 3], stride=[1, 1],

                  padding='same', data_format='channels_first'):
    data_format='channels_last'
    if data_format == 'channels_first':
        input_n_filters = input_layer.get_shape().as_list()[1]
    else:
        input_n_filters = input_layer.get_shape().as_list()[3]

    if out_n_filters != input_n_filters:
        skip_layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(
            input_layer)
    else:
        skip_layer = input_layer

    layer = skip_layer
    for j in range(2):

        for i in range(2):
            if i == 0:

                layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                    layer)
                if batch_normalization:
                    layer1 = BatchNormalization()(layer1)
                layer1 = Activation('relu')(layer1)
            layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                add([layer1, layer]))
            if batch_normalization:
                layer1 = BatchNormalization()(layer1)
            layer1 = Activation('relu')(layer1)
        layer = layer1

    out_layer = add([layer, skip_layer])
    return out_layer

In [ ]:
def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same",use_bias=False,kernel_initializer='he_normal')(y1)
    y1 = Lambda(mvn)(y1)
    y1 = ReLU()(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
    y2 = Lambda(mvn)(y2)
    y2 = ReLU()(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
    y3 = Lambda(mvn)(y3)
    y3 = ReLU()(y3)

    y4 = Conv2D(filter, 5, dilation_rate=12, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
    y4 = Lambda(mvn)(y4)
    y4 = ReLU()(y4)

    y5 = Conv2D(filter, 7, dilation_rate=18, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
    y5 = Lambda(mvn)(y5)
    y5 = ReLU()(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False,kernel_initializer='he_normal')(y)
    y = Lambda(mvn)(y)
    y = ReLU()(y)
    return y

In [ ]:
def refinement_module(x, input_filter, middle_filter):
  hx = x
  hx0 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx)
  #print(hx0.shape)

  hx1 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx0)
  # hx1 = Lambda(mvn)(hx1)
  hx1 = BatchNormalization()(hx1)
  hx1 = ReLU()(hx1)
  hx1 = MaxPooling2D(strides=(2,2))(hx1)
  #print(hx1.shape)

  hx2 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx1)
  # hx2 = Lambda(mvn)(hx2)
  hx2 = BatchNormalization()(hx2)
  hx2 = ReLU()(hx2)
  hx2 = MaxPooling2D(strides=(2,2))(hx2)
  #print(hx2.shape)

  hx3 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx2)
  # hx3 = Lambda(mvn)(hx3)
  hx3 = BatchNormalization()(hx3)
  hx3 = ReLU()(hx3)
  hx3 = MaxPooling2D(strides=(2,2))(hx3)
  #print(hx3.shape)

  hx4 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx3)
  # hx4 = Lambda(mvn)(hx4)
  hx4 = BatchNormalization()(hx4)
  hx4 = ReLU()(hx4)
  hx4 = MaxPooling2D(strides=(2,2))(hx4)
  #print(hx4.shape)

  hx5 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx4)
  # hx5 = Lambda(mvn)(hx5)
  hx5 = BatchNormalization()(hx5)
  hx5 = ReLU()(hx5)
  #print(hx5.shape)
  hx5 = UpSampling2D(size=(2,2), interpolation='bilinear')(hx5)
  #print(hx5.shape)

  hx6 = Concatenate()([hx5, hx3])
  hx6 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx6)
  # hx6 = Lambda(mvn)(hx6)
  hx6 = BatchNormalization()(hx6)
  hx6 = ReLU()(hx6)
  hx6 = UpSampling2D(size=(2,2), interpolation='bilinear')(hx6)

  hx7 = Concatenate()([hx6, hx2])
  hx7 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx7)
  # hx7 = Lambda(mvn)(hx7)
  hx7 = BatchNormalization()(hx7)
  hx7 = ReLU()(hx7)
  hx7 = UpSampling2D(size=(2,2), interpolation='bilinear')(hx7)

  hx8 = Concatenate()([hx7, hx1])
  hx6 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx8)
  # hx8 = Lambda(mvn)(hx8)
  hx8 = BatchNormalization()(hx8)
  hx8 = ReLU()(hx8)
  hx8 = UpSampling2D(size=(2,2), interpolation='bilinear')(hx8)

  hx9 = Concatenate()([hx8, hx0])
  hx9 = Conv2D(middle_filter*1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx9)
  # hx9 = Lambda(mvn)(hx9)
  hx9 = BatchNormalization()(hx9)
  hx9 = ReLU()(hx9)
  #hx9 = UpSampling2D((hx0.shape[1], hx0.shape[2]), interpolation='bilinear')(hx9)

  hx10 = Conv2D(1, 3, padding="same", use_bias=False,kernel_initializer='he_normal')(hx9)
  return hx10+x

In [ ]:
def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = Lambda(mvn)(x) #BatchNormalization()(x)
    if activation == True:
        x =  ReLU()(x) #LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = LeakyReLU(alpha=0.1)(blockInput)
    x = BatchNormalization()(x)
    blockInput = BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3))
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    return x

def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), activation='relu', name=None):
    '''
    2D Convolutional layers
    
    Arguments:
        x {keras layer} -- input layer 
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters
    
    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(1, 1)})
        activation {str} -- activation function (default: {'relu'})
        name {str} -- name of the layer (default: {None})
    
    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2D(filters, (num_row, num_col), strides=strides, padding=padding, use_bias=False)(x)
    x = Lambda(mvn)(x) #BatchNormalization(axis=3, scale=False)(x)

    if(activation == None):
        return x

    x = Activation(activation, name=name)(x)

    return x

def ResPath(filters, length, inp):
    '''
    ResPath
    
    Arguments:
        filters {int} -- [description]
        length {int} -- length of ResPath
        inp {keras layer} -- input layer 
    
    Returns:
        [keras layer] -- [output layer]
    '''

    shortcut = inp
    shortcut = conv2d_bn(shortcut, filters, 1, 1, activation=None, padding='same')

    out = conv2d_bn(inp, filters, 3, 3, activation='relu', padding='same')

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = Lambda(mvn)(out) #BatchNormalization(axis=3)(out)

    for i in range(length-1):

        shortcut = out
        shortcut = conv2d_bn(shortcut, filters, 1, 1, activation=None, padding='same')

        out = conv2d_bn(out, filters, 3, 3, activation='relu', padding='same')

        out = add([shortcut, out])
        out = Activation('relu')(out)
        out = Lambda(mvn)(out) #BatchNormalization(axis=3)(out)

    return out

In [ ]:
def _convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=tf.keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = _convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = _convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = _convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = _convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = _convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = _convolution_block(x, kernel_size=1)
    return output

In [ ]:
class BilinearUpsampling(Layer):
    """Just a simple bilinear upsampling layer. Works only with TF.
       Args:
           upsampling: tuple of 2 numbers > 0. The upsampling ratio for h and w
           output_size: used instead of upsampling arg if passed!
    """

    def __init__(self, upsampling=(2, 2), output_size=None, data_format=None, **kwargs):

        super(BilinearUpsampling, self).__init__(**kwargs)

        #self.data_format = K.normalize_data_format(data_format)
        self.data_format = None
        self.input_spec = InputSpec(ndim=4)
        if output_size:
            self.output_size = conv_utils.normalize_tuple(
                output_size, 2, 'output_size')
            self.upsampling = None
        else:
            self.output_size = None
            self.upsampling = conv_utils.normalize_tuple(
                upsampling, 2, 'upsampling')

    def compute_output_shape(self, input_shape):
        if self.upsampling:
            height = self.upsampling[0] * \
                input_shape[1] if input_shape[1] is not None else None
            width = self.upsampling[1] * \
                input_shape[2] if input_shape[2] is not None else None
        else:
            height = self.output_size[0]
            width = self.output_size[1]
        return (input_shape[0],
                height,
                width,
                input_shape[3])

    def call(self, inputs):
        if self.upsampling:
            return K.tf.image.resize_bilinear(inputs, (inputs.shape[1] * self.upsampling[0],
                                                       inputs.shape[2] * self.upsampling[1]),
                                              align_corners=True)
        else:
            return K.tf.image.resize_bilinear(inputs, (self.output_size[0],
                                                       self.output_size[1]),
                                              align_corners=True)

    def get_config(self):
        config = {'upsampling': self.upsampling,
                  'output_size': self.output_size,
                  'data_format': self.data_format}
        base_config = super(BilinearUpsampling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def SepConv_BN(x, filters, prefix, stride=1, kernel_size=3, rate=1, depth_activation=False, epsilon=1e-3):
    """ SepConv with BN between depthwise & pointwise. Optionally add activation after BN
        Implements right "same" padding for even kernel sizes
        Args:
            x: input tensor
            filters: num of filters in pointwise convolution
            prefix: prefix before name
            stride: stride at depthwise conv
            kernel_size: kernel size for depthwise convolution
            rate: atrous rate for depthwise convolution
            depth_activation: flag to use activation between depthwise & poinwise convs
            epsilon: epsilon to use in BN layer
    """

    if stride == 1:
        depth_padding = 'same'
    else:
        kernel_size_effective = kernel_size + (kernel_size - 1) * (rate - 1)
        pad_total = kernel_size_effective - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        x = ZeroPadding2D((pad_beg, pad_end))(x)
        depth_padding = 'valid'

    if not depth_activation:
        x = Activation('relu')(x)
    x = DepthwiseConv2D((kernel_size, kernel_size), strides=(stride, stride), dilation_rate=(rate, rate),
                        padding=depth_padding, use_bias=False, name=prefix + '_depthwise')(x)
    x = BatchNormalization(name=prefix + '_depthwise_BN', epsilon=epsilon)(x)
    if depth_activation:
        x = Activation('relu')(x)
    x = Conv2D(filters, (1, 1), padding='same',
               use_bias=False, name=prefix + '_pointwise')(x)
    x = BatchNormalization(name=prefix + '_pointwise_BN', epsilon=epsilon)(x)
    if depth_activation:
        x = Activation('relu')(x)

    return x


def _conv2d_same(x, filters, prefix, stride=1, kernel_size=3, rate=1):
    """Implements right 'same' padding for even kernel sizes
        Without this there is a 1 pixel drift when stride = 2
        Args:
            x: input tensor
            filters: num of filters in pointwise convolution
            prefix: prefix before name
            stride: stride at depthwise conv
            kernel_size: kernel size for depthwise convolution
            rate: atrous rate for depthwise convolution
    """
    if stride == 1:
        return Conv2D(filters,
                      (kernel_size, kernel_size),
                      strides=(stride, stride),
                      padding='same', use_bias=False,
                      dilation_rate=(rate, rate),
                      name=prefix)(x)
    else:
        kernel_size_effective = kernel_size + (kernel_size - 1) * (rate - 1)
        pad_total = kernel_size_effective - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        x = ZeroPadding2D((pad_beg, pad_end))(x)
        return Conv2D(filters,
                      (kernel_size, kernel_size),
                      strides=(stride, stride),
                      padding='valid', use_bias=False,
                      dilation_rate=(rate, rate),
                      name=prefix)(x)


def _xception_block(inputs, depth_list, prefix, skip_connection_type, stride,
                    rate=1, depth_activation=False, return_skip=False):
    """ Basic building block of modified Xception network
        Args:
            inputs: input tensor
            depth_list: number of filters in each SepConv layer. len(depth_list) == 3
            prefix: prefix before name
            skip_connection_type: one of {'conv','sum','none'}
            stride: stride at last depthwise conv
            rate: atrous rate for depthwise convolution
            depth_activation: flag to use activation between depthwise & pointwise convs
            return_skip: flag to return additional tensor after 2 SepConvs for decoder
            """
    residual = inputs
    for i in range(3):
        residual = SepConv_BN(residual,
                              depth_list[i],
                              prefix + '_separable_conv{}'.format(i + 1),
                              stride=stride if i == 2 else 1,
                              rate=rate,
                              depth_activation=depth_activation)
        if i == 1:
            skip = residual
    if skip_connection_type == 'conv':
        shortcut = _conv2d_same(inputs, depth_list[-1], prefix + '_shortcut',
                                kernel_size=1,
                                stride=stride)
        shortcut = BatchNormalization(name=prefix + '_shortcut_BN')(shortcut)
        outputs = layers.add([residual, shortcut])
    elif skip_connection_type == 'sum':
        outputs = layers.add([residual, inputs])
    elif skip_connection_type == 'none':
        outputs = residual
    if return_skip:
        return outputs, skip
    else:
        return outputs


def relu6(x):
    return K.relu(x, max_value=6)


def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def _inverted_res_block(inputs, expansion, stride, alpha, filters, block_id, skip_connection, rate=1):
    in_channels = inputs._keras_shape[-1]
    pointwise_conv_filters = int(filters * alpha)
    pointwise_filters = _make_divisible(pointwise_conv_filters, 8)
    x = inputs
    prefix = 'expanded_conv_{}_'.format(block_id)
    if block_id:
        # Expand

        x = Conv2D(expansion * in_channels, kernel_size=1, padding='same',
                   use_bias=False, activation=None,
                   name=prefix + 'expand')(x)
        x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                               name=prefix + 'expand_BN')(x)
        x = Activation(relu6, name=prefix + 'expand_relu')(x)
    else:
        prefix = 'expanded_conv_'
    # Depthwise
    x = DepthwiseConv2D(kernel_size=3, strides=stride, activation=None,
                        use_bias=False, padding='same', dilation_rate=(rate, rate),
                        name=prefix + 'depthwise')(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                           name=prefix + 'depthwise_BN')(x)

    x = Activation(relu6, name=prefix + 'depthwise_relu')(x)

    # Project
    x = Conv2D(pointwise_filters,
               kernel_size=1, padding='same', use_bias=False, activation=None,
               name=prefix + 'project')(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                           name=prefix + 'project_BN')(x)

    if skip_connection:
        return Add(name=prefix + 'add')([inputs, x])

    # if in_channels == pointwise_filters and stride == 1:
    #    return Add(name='res_connect_' + str(block_id))([inputs, x])

    return x

In [ ]:
def Deeplabv3(weights='pascal_voc', input_tensor=None, input_shape=(512, 512, 3), classes=21, backbone='mobilenetv2', OS=16, alpha=1.):
    """ Instantiates the Deeplabv3+ architecture

    Optionally loads weights pre-trained
    on PASCAL VOC. This model is available for TensorFlow only,
    and can only be used with inputs following the TensorFlow
    data format `(width, height, channels)`.
    # Arguments
        weights: one of 'pascal_voc' (pre-trained on pascal voc)
            or None (random initialization)
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: shape of input image. format HxWxC
            PASCAL VOC model was trained on (512,512,3) images
        classes: number of desired classes. If classes != 21,
            last layer is initialized randomly
        backbone: backbone to use. one of {'xception','mobilenetv2'}
        OS: determines input_shape/feature_extractor_output ratio. One of {8,16}.
            Used only for xception backbone.
        alpha: controls the width of the MobileNetV2 network. This is known as the
            width multiplier in the MobileNetV2 paper.
                - If `alpha` < 1.0, proportionally decreases the number
                    of filters in each layer.
                - If `alpha` > 1.0, proportionally increases the number
                    of filters in each layer.
                - If `alpha` = 1, default number of filters from the paper
                    are used at each layer.
            Used only for mobilenetv2 backbone

    # Returns
        A Keras model instance.

    # Raises
        RuntimeError: If attempting to run this model with a
            backend that does not support separable convolutions.
        ValueError: in case of invalid argument for `weights` or `backbone`

    """

    if not (weights in {'pascal_voc', None}):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `pascal_voc` '
                         '(pre-trained on PASCAL VOC)')

    if K.backend() != 'tensorflow':
        raise RuntimeError('The Deeplabv3+ model is only available with '
                           'the TensorFlow backend.')

    if not (backbone in {'xception', 'mobilenetv2'}):
        raise ValueError('The `backbone` argument should be either '
                         '`xception`  or `mobilenetv2` ')

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    if backbone == 'xception':
        if OS == 8:
            entry_block3_stride = 1
            middle_block_rate = 2  # ! Not mentioned in paper, but required
            exit_block_rates = (2, 4)
            atrous_rates = (12, 24, 36)
        else:
            entry_block3_stride = 2
            middle_block_rate = 1
            exit_block_rates = (1, 2)
            atrous_rates = (6, 12, 18)

        x = Conv2D(32, (3, 3), strides=(2, 2),
                   name='entry_flow_conv1_1', use_bias=False, padding='same')(img_input)
        x = BatchNormalization(name='entry_flow_conv1_1_BN')(x)
        x = Activation('relu')(x)

        x = _conv2d_same(x, 64, 'entry_flow_conv1_2', kernel_size=3, stride=1)
        x = BatchNormalization(name='entry_flow_conv1_2_BN')(x)
        x = Activation('relu')(x)

        x = _xception_block(x, [128, 128, 128], 'entry_flow_block1',
                            skip_connection_type='conv', stride=2,
                            depth_activation=False)
        x, skip1 = _xception_block(x, [256, 256, 256], 'entry_flow_block2',
                                   skip_connection_type='conv', stride=2,
                                   depth_activation=False, return_skip=True)

        x = _xception_block(x, [728, 728, 728], 'entry_flow_block3',
                            skip_connection_type='conv', stride=entry_block3_stride,
                            depth_activation=False)
        for i in range(16):
            x = _xception_block(x, [728, 728, 728], 'middle_flow_unit_{}'.format(i + 1),
                                skip_connection_type='sum', stride=1, rate=middle_block_rate,
                                depth_activation=False)

        x = _xception_block(x, [728, 1024, 1024], 'exit_flow_block1',
                            skip_connection_type='conv', stride=1, rate=exit_block_rates[0],
                            depth_activation=False)
        x = _xception_block(x, [1536, 1536, 2048], 'exit_flow_block2',
                            skip_connection_type='none', stride=1, rate=exit_block_rates[1],
                            depth_activation=True)

    else:
        OS = 8
        first_block_filters = _make_divisible(32 * alpha, 8)
        x = Conv2D(first_block_filters,
                   kernel_size=3,
                   strides=(2, 2), padding='same',
                   use_bias=False, name='Conv')(img_input)
        x = BatchNormalization(
            epsilon=1e-3, momentum=0.999, name='Conv_BN')(x)
        x = Activation(relu6, name='Conv_Relu6')(x)

        x = _inverted_res_block(x, filters=16, alpha=alpha, stride=1,
                                expansion=1, block_id=0, skip_connection=False)

        x = _inverted_res_block(x, filters=24, alpha=alpha, stride=2,
                                expansion=6, block_id=1, skip_connection=False)
        x = _inverted_res_block(x, filters=24, alpha=alpha, stride=1,
                                expansion=6, block_id=2, skip_connection=True)

        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=2,
                                expansion=6, block_id=3, skip_connection=False)
        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                                expansion=6, block_id=4, skip_connection=True)
        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                                expansion=6, block_id=5, skip_connection=True)

        # stride in block 6 changed from 2 -> 1, so we need to use rate = 2
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1,  # 1!
                                expansion=6, block_id=6, skip_connection=False)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=7, skip_connection=True)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=8, skip_connection=True)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=9, skip_connection=True)

        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=10, skip_connection=False)
        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=11, skip_connection=True)
        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=12, skip_connection=True)

        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=2,  # 1!
                                expansion=6, block_id=13, skip_connection=False)
        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=14, skip_connection=True)
        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=15, skip_connection=True)

        x = _inverted_res_block(x, filters=320, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=16, skip_connection=False)

    # end of feature extractor

    # branching for Atrous Spatial Pyramid Pooling

    # Image Feature branch
    #out_shape = int(np.ceil(input_shape[0] / OS))
    b4 = AveragePooling2D(pool_size=(int(np.ceil(input_shape[0] / OS)), int(np.ceil(input_shape[1] / OS))))(x)
    b4 = Conv2D(256, (1, 1), padding='same',
                use_bias=False, name='image_pooling')(b4)
    b4 = BatchNormalization(name='image_pooling_BN', epsilon=1e-5)(b4)
    b4 = Activation('relu')(b4)
    b4 = BilinearUpsampling((int(np.ceil(input_shape[0] / OS)), int(np.ceil(input_shape[1] / OS))))(b4)

    # simple 1x1
    b0 = Conv2D(256, (1, 1), padding='same', use_bias=False, name='aspp0')(x)
    b0 = BatchNormalization(name='aspp0_BN', epsilon=1e-5)(b0)
    b0 = Activation('relu', name='aspp0_activation')(b0)

    # there are only 2 branches in mobilenetV2. not sure why
    if backbone == 'xception':
        # rate = 6 (12)
        b1 = SepConv_BN(x, 256, 'aspp1',
                        rate=atrous_rates[0], depth_activation=True, epsilon=1e-5)
        # rate = 12 (24)
        b2 = SepConv_BN(x, 256, 'aspp2',
                        rate=atrous_rates[1], depth_activation=True, epsilon=1e-5)
        # rate = 18 (36)
        b3 = SepConv_BN(x, 256, 'aspp3',
                        rate=atrous_rates[2], depth_activation=True, epsilon=1e-5)

        # concatenate ASPP branches & project
        x = Concatenate()([b4, b0, b1, b2, b3])
    else:
        x = Concatenate()([b4, b0])

    x = Conv2D(256, (1, 1), padding='same',
               use_bias=False, name='concat_projection')(x)
    x = BatchNormalization(name='concat_projection_BN', epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Dropout(0.1)(x)

    # DeepLab v.3+ decoder

    if backbone == 'xception':
        # Feature projection
        # x4 (x2) block
        x = BilinearUpsampling(output_size=(int(np.ceil(input_shape[0] / 4)),
                                            int(np.ceil(input_shape[1] / 4))))(x)
        dec_skip1 = Conv2D(48, (1, 1), padding='same',
                           use_bias=False, name='feature_projection0')(skip1)
        dec_skip1 = BatchNormalization(
            name='feature_projection0_BN', epsilon=1e-5)(dec_skip1)
        dec_skip1 = Activation('relu')(dec_skip1)
        x = Concatenate()([x, dec_skip1])
        x = SepConv_BN(x, 256, 'decoder_conv0',
                       depth_activation=True, epsilon=1e-5)
        x = SepConv_BN(x, 256, 'decoder_conv1',
                       depth_activation=True, epsilon=1e-5)

    # you can use it with arbitary number of classes
    if classes == 21:
        last_layer_name = 'logits_semantic'
    else:
        last_layer_name = 'custom_logits_semantic'

    x = Conv2D(classes, (1, 1), padding='same', name=last_layer_name)(x)
    x = BilinearUpsampling(output_size=(input_shape[0], input_shape[1]))(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, x, name='deeplabv3+')

    # load weights

    if weights == 'pascal_voc':
        if backbone == 'xception':
            weights_path = get_file('deeplabv3_xception_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH_X,
                                    cache_subdir='models')
        else:
            weights_path = get_file('deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH_MOBILE,
                                    cache_subdir='models')
        model.load_weights(weights_path, by_name=True)
    return model

In [ ]:
class Swish(tf.keras.layers.Layer):
    def __init__(self, name=None, **kwargs):
        super().__init__(name=name, **kwargs)

    def call(self, inputs, **kwargs):
        return tf.nn.swish(inputs)

    def get_config(self):
        config = super().get_config()
        config['name'] = self.name
        return config

def squeeze_excite_block(reduce_ratio=0.25,name_block=None):
  def call(inputs):
    filters = inputs.shape[-1]
    num_reduced_filters= max(1, int(filters * reduce_ratio))
    se = Lambda(lambda a: K.mean(a, axis=[1,2], keepdims=True))(inputs)

    se = Conv2D(
            num_reduced_filters,
            kernel_size=[1, 1],
            strides=[1, 1],
            kernel_initializer='he_normal',
            padding='same',
            use_bias=True
        )(se)
    se = Swish()(se)
    se = Conv2D(
            filters,
            kernel_size=[1, 1],
            strides=[1, 1],
            kernel_initializer='he_normal',
            padding='same',
            use_bias=True
        )(se)
    se = Activation('sigmoid')(se)
    if name_block is not None:
      out = Multiply(name=name_block)([se, inputs])
    else : 
      out = Multiply()([se, inputs])
    return out
  return call

def conv_block(filters,block_name=None): #kernel_size = (3,3), dilation = 1
  def call(inputs):
    x = inputs

    x = Conv2D(filters, kernel_size=(3,3), padding="same",dilation_rate =1 ,use_bias=False,kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Swish()(x)

    x = Conv2D(filters, kernel_size=(3,3), padding="same",dilation_rate =1, use_bias=False,kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Swish()(x)

    x = squeeze_excite_block(name_block=block_name)(x)

    return x
  return call

In [ ]:
def _ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same",use_bias=False,kernel_initializer='he_normal')(y1)
    y1 = BatchNormalization()(y1)
    y1 = Swish()(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)
    y1 = squeeze_excite_block()(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
    y2 = BatchNormalization()(y2)
    y2 = Swish()(y2)
    y2 = squeeze_excite_block()(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
    y3 = BatchNormalization()(y3)
    y3 = Swish()(y3)
    y3 = squeeze_excite_block()(y3)

    y4 = Conv2D(filter, 5, dilation_rate=12, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
    y4 = BatchNormalization()(y4)
    y4 = Swish()(y4)
    y4 = squeeze_excite_block()(y4)

    y5 = Conv2D(filter, 7, dilation_rate=18, padding="same", use_bias=False,kernel_initializer='he_normal')(x)
    y5 = BatchNormalization()(y5)
    y5 = Swish()(y5)
    y5 = squeeze_excite_block()(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False,kernel_initializer='he_normal')(y)
    y = BatchNormalization()(y)
    y = Swish()(y)
    y = squeeze_excite_block()(y)
    return y

In [ ]:
def decoder_block(n_filter,skip=None):
  def call(inputs):
    x= Conv2DTranspose(n_filter, (2,2), strides=(2, 2), padding='same',kernel_initializer = 'he_normal')(inputs)
    out = x
    if skip is not None :
      attention = conv_block(n_filter)(skip)
      out = Concatenate()([x,attention])
    out = Dropout(0.5)(out)
    out = conv_block(n_filter)(out)

    return out
  return call
  
def dow_block(kernel_size=(2,2),stride=(2,2)):
  def call(inputs):
    out = MaxPooling2D(kernel_size, strides=stride)(inputs)
    return out
  return call

In [ ]:
def encoderSegnet(input_s=(128,128,1)):
  inp= Input(shape=input_s)
  o = inp
  nums_filter=[64,128,256,512,512]
  count=0
  for f in nums_filter[:-1]:
    count+=1
    o = conv_block(f,block_name='output_block_'+str(count))(o) 
    o = dow_block()(o)

  o = conv_block(nums_filter[-1],block_name='output_block_'+str(count+1))(o)
  #o = Dropout(0.5)(o)
  return Model(inp,o)

list_skip = ["output_block_4", "output_block_3", "output_block_2", "output_block_1"]

In [ ]:
def expend_as(tensor, rep):
    my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep})(tensor)
    return my_repeat

def SalientAttentionBlock(f_maps, sal_ins, pool_maps, num_fmaps):
    # Inputs: feature maps from UNet, saliency images, pooled layers from UNet, number of output feature maps
    conv1_salins = Conv2D(128, (1, 1), activation='relu')(sal_ins)
    conv1_fmaps = Conv2D(128, (1, 1), strides=(2, 2), activation='relu')(f_maps)
    attn_add = add([conv1_fmaps,conv1_salins])
    conv_1d = Conv2D(128, (3, 3), activation='relu', padding='same')(attn_add)
    conv_1d = Conv2D(128, (3, 3), activation='relu', padding='same')(conv_1d)
    conv_1d = Conv2D(1, (1, 1), activation='relu')(conv_1d)
    conv_1d = expend_as(conv_1d,32)
    conv_nd = Conv2D(num_fmaps, (1, 1), activation='relu')(conv_1d)
    attn_act = Activation('sigmoid')(conv_nd)
    attn = multiply([attn_act, pool_maps])
    return attn

def UNetBlock(in_fmaps, num_fmaps):
    # Inputs: feature maps for UNet, number of output feature maps
    conv1 = Conv2D(num_fmaps, (3, 3), activation='relu', padding='same')(in_fmaps)
    conv_out = Conv2D(num_fmaps, (3, 3), activation='relu', padding='same')(conv1)
    return conv_out

In [ ]:
def _squeeze_excite_block_(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x

def _conv_block_(inputs, filters):
    x = inputs

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = _squeeze_excite_block_(x)

    return x

def encoder1(inputs):
    skip_connections = []

    model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_tensor=inputs)
    names = ["block1_conv2", "block2_conv2", "block3_conv4", "block4_conv4"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("block5_conv4").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [256, 128, 64, 32]
    skip_connections.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_connections[i]])
        x = _conv_block_(x, f)

    return x

def encoder2(inputs):
    num_filters = [32, 64, 128, 256]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        x = _conv_block_(x, f)
        skip_connections.append(x)
        x = MaxPool2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 32]
    skip_2.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_1[i], skip_2[i]])
        x = _conv_block_(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, (1, 1), padding="same")(inputs)
    x = Activation('sigmoid')(x)
    return x

def Upsample(tensor, size):
    """Bilinear upsampling"""
    def _upsample(x, size):
        return tf.image.resize(images=x, size=size)
    return Lambda(lambda x: _upsample(x, size), output_shape=size)(tensor)

def _ASPP_(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

## Attention U-Net

In [ ]:
'''input_shape = (256,256,3)
data = Input(shape=input_shape, dtype='float', name='data')

mvn0 = BatchNormalization()(data)
conv1 = Conv2D(64, 3, padding = 'same')(mvn0)
conv1 = BatchNormalization()(conv1)
conv1 = Activation('relu')(conv1)
conv1 = Conv2D(64, 3,  padding = 'same')(conv1)
conv1 = BatchNormalization()(conv1)
conv1 = Activation('relu')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3,  padding = 'same')(pool1)
conv2 = BatchNormalization()(conv2)
conv2 = Activation('relu')(conv2)
conv2 = Conv2D(128, 3,  padding = 'same')(conv2)
conv2 = BatchNormalization()(conv2)
conv2 = Activation('relu')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3,  padding = 'same')(pool2)
conv3 = BatchNormalization()(conv3)
conv3 = Activation('relu')(conv3)
conv3 = Conv2D(256, 3,  padding = 'same')(conv3)
conv3 = BatchNormalization()(conv3)
conv3 = Activation('relu')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
conv4 = Conv2D(512, 3,  padding = 'same')(pool3)
conv4 = BatchNormalization()(conv4)
conv4 = Activation('relu')(conv4)
conv4 = Conv2D(512, 3,  padding = 'same')(conv4)
conv4 = BatchNormalization()(conv4)
conv4 = Activation('relu')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
#pool4 = ASPP(pool4,1024)

conv5 = Conv2D(1024, 3,  padding = 'same')(pool4)
conv5 = BatchNormalization()(conv5)
conv5 = Activation('relu')(conv5)
conv5 = Conv2D(1024, 3,  padding = 'same')(conv5)
conv5 = BatchNormalization()(conv5)
conv5 = Activation('relu')(conv5)
drop5 = Dropout(0.5)(conv5)

merge6 = attention_up_and_concate(conv5,conv4)
conv6 = Conv2D(512, 3,  padding = 'same')(merge6)
conv6 = BatchNormalization()(conv6)
conv6 = Activation('relu')(conv6)
conv6 = Conv2D(512, 3,  padding = 'same')(conv6)
conv6 = BatchNormalization()(conv6)
conv6 = Activation('relu')(conv6)

merge7 = attention_up_and_concate(conv6,conv3)
conv7 = Conv2D(256, 3,  padding = 'same')(merge7)
conv7 = BatchNormalization()(conv7)
conv7 = Activation('relu')(conv7)
conv7 = Conv2D(256, 3,  padding = 'same')(conv7)
conv7 = BatchNormalization()(conv7)
conv7 = Activation('relu')(conv7)

merge8 = attention_up_and_concate(conv7,conv2)
conv8 = Conv2D(128, 3,  padding = 'same')(merge8)
conv8 = BatchNormalization()(conv8)
conv8 = Activation('relu')(conv8)
conv8 = Conv2D(128, 3,  padding = 'same')(conv8)
conv8 = BatchNormalization()(conv8)
conv8 = Activation('relu')(conv8)

merge9 = attention_up_and_concate(conv8,conv1)
conv9 = Conv2D(64, 3,  padding = 'same')(merge9)
conv9 = BatchNormalization()(conv9)
conv9 = Activation('relu')(conv9)
conv9 = Conv2D(64, 3,  padding = 'same')(conv9)
conv9 = BatchNormalization()(conv9)
conv9 = Activation('relu')(conv9)
conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

model = Model(data, conv10) 
model.summary()'''
#0.9043

## FCN

In [ ]:
'''input_shape = (256,256,3)
inputs = Input(shape=input_shape, dtype='float', name='data')

mvn0 = Lambda(mvn)(inputs)
pad0 = ZeroPadding2D(padding = 5)(mvn0)

conv1 = Conv2D(64, kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(pad0)
mvn1 = Lambda(mvn)(conv1)

conv2 = Conv2D(64,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias =True)(mvn1)
mvn2 = Lambda(mvn)(conv2)

conv3 = Conv2D(64,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias =True)(mvn2)
mvn3 = Lambda(mvn)(conv3)

mxp1 = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'valid')(mvn3)

conv4 = Conv2D(128,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mxp1)
mvn4 = Lambda(mvn)(conv4)

conv5 = Conv2D(128,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mvn4)
mvn5 = Lambda(mvn)(conv5)

conv6 = Conv2D(128,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mvn5)
mvn6 = Lambda(mvn)(conv6)

conv7 = Conv2D(128,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mvn6)
mvn7 = Lambda(mvn)(conv7)

#drop1 = Dropout(rate = 0.5)(mvn7)

mxp2 = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'valid')(mvn7)

conv8 = Conv2D(256,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mxp2)
mvn8 = Lambda(mvn)(conv8)

conv9 = Conv2D(256,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mvn8)
mvn9 = Lambda(mvn)(conv9)

conv10 = Conv2D(256,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mvn9)
mvn10 = Lambda(mvn)(conv10)

conv11 = Conv2D(256,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mvn10)
mvn11 = Lambda(mvn)(conv11)

mxp3 = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'valid')(mvn11)
    
drop2 = Dropout(rate = 0.5)(mxp3)

conv12 = Conv2D(512,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(drop2)
mvn12 = Lambda(mvn)(conv12)

conv13 = Conv2D(512,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mvn12)
mvn13 = Lambda(mvn)(conv13)

conv14 = Conv2D(512,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mvn13)
mvn14 = Lambda(mvn)(conv14)

conv15 = Conv2D(512,kernel_size = (3,3), padding = 'same', activation = 'relu',  use_bias = True)(mvn14)
mvn15 = Lambda(mvn)(conv15)

drop3 = Dropout(rate = 0.5)(mvn15)

score_conv15 = Conv2D(1, (1,1), strides = (1,1), use_bias = True, padding = 'valid')(drop3)

upsample1 = Conv2DTranspose(1, (3,3), strides = (2,2), use_bias = False, padding = 'valid')(score_conv15)
#Conv2DTranspose = Deconvolution : phép biến đổi đi ngược lại tích chập 
#từ một thứ có hình dạng đầu ra của 1 phép tích chập sang 1 thứ có hình dạng đầu vào của nó.
score_conv11 = Conv2D(1, (1,1), strides = (1,1), use_bias = True, padding = 'valid')(mvn11)

crop1 = Lambda(crop)([upsample1, score_conv11])
fuse1 = average([crop1, upsample1])

upsample2 = Conv2DTranspose(1, (3,3), strides = (2,2), use_bias = False, padding = 'valid')(fuse1)

score_conv7 = Conv2D(1, (1,1), strides = (1,1), use_bias = True, padding = 'valid')(mvn7)

crop2 = Lambda(crop)([upsample2, score_conv7])
fuse2 = average([crop2, upsample2])

upsample3 = Conv2DTranspose(1, (3,3), strides = (2,2), use_bias = False, padding = 'valid' )(fuse2)

crop3 = Lambda(crop)([inputs, upsample3])
    
predict = Conv2D(1, (1,1), strides = (1,1), padding = 'valid', activation = 'sigmoid', use_bias = True)(crop3)
model1 = Model(inputs=inputs, outputs=predict)

model1.summary()'''

## UNet with EfficientNet encoder and Residual decoder

In [ ]:
'''input_shape = (256,256,3)
dropout_rate = 0.5

backbone = EfficientNetB4(weights='imagenet', include_top=False, input_shape=input_shape)
input = backbone.input
start_neurons = 16

conv4 = backbone.layers[342].output
conv4 = Lambda(mvn)(conv4)
conv4 = ReLU()(conv4) #LeakyReLU(alpha=0.1)(conv4)
pool4 = MaxPooling2D((2, 2))(conv4)
pool4 = ASPP(pool4,start_neurons * 32)
pool4 = Dropout(dropout_rate)(pool4)
    
# Middle
convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
convm = residual_block(convm,start_neurons * 32)
convm = residual_block(convm,start_neurons * 32)
convm = Lambda(mvn)(convm)
convm = ReLU()(convm) #LeakyReLU(alpha=0.1)(convm)
    
deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
uconv4 = concatenate([deconv4, conv4])
uconv4 = Dropout(dropout_rate)(uconv4)
    
uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
uconv4 = residual_block(uconv4,start_neurons * 16)
uconv4 = residual_block(uconv4,start_neurons * 16)
uconv4 = Lambda(mvn)(uconv4)
uconv4 = ReLU()(uconv4) #LeakyReLU(alpha=0.1)(uconv4)
    
deconv3 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
conv3 = backbone.layers[154].output
uconv3 = concatenate([deconv3, conv3])    
uconv3 = Dropout(dropout_rate)(uconv3)
    
uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
uconv3 = residual_block(uconv3,start_neurons * 8)
uconv3 = residual_block(uconv3,start_neurons * 8)
uconv3 = Lambda(mvn)(uconv3)
uconv3 = ReLU()(uconv3) #LeakyReLU(alpha=0.1)(uconv3)

deconv2 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
conv2 = backbone.layers[94].output
uconv2 = concatenate([deconv2, conv2])
        
uconv2 = Dropout(0.1)(uconv2)
uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
uconv2 = residual_block(uconv2,start_neurons * 4)
uconv2 = residual_block(uconv2,start_neurons * 4)
uconv2 = Lambda(mvn)(uconv2)
uconv2 = ReLU()(uconv2) #LeakyReLU(alpha=0.1)(uconv2)
    
deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
conv1 = backbone.layers[30].output
uconv1 = concatenate([deconv1, conv1])
    
uconv1 = Dropout(0.1)(uconv1)
uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
uconv1 = residual_block(uconv1,start_neurons * 2)
uconv1 = residual_block(uconv1,start_neurons * 2)
uconv1 = Lambda(mvn)(uconv1)
uconv1 = ReLU()(uconv1) #LeakyReLU(alpha=0.1)(uconv1)
    
uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
uconv0 = Dropout(0.1)(uconv0)
uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
uconv0 = residual_block(uconv0,start_neurons * 1)
uconv0 = residual_block(uconv0,start_neurons * 1)
uconv0 = Lambda(mvn)(uconv0)
uconv0 = ReLU()(uconv0) #LeakyReLU(alpha=0.1)(uconv0)

uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv0)  
uconv0 = Dropout(dropout_rate/2)(uconv0)
output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    
    
model = Model(input, output_layer)
model.summary()'''
#0.9150 

## Proposed Model with BatchNorm

In [ ]:
def ARM(x):
  gp1 = AveragePooling2D(pool_size = x.shape[1])(x)
  gp = Conv2D(x.shape[-1],1,padding = 'valid', kernel_initializer='he_normal')(gp1)
  gp = BatchNormalization()(gp)
  gp =  Activation('sigmoid')(gp)
  return Multiply()([gp, x])

In [ ]:
def BGA(uconv, econv):
     # econv = ARM(econv)
     a1 = DepthwiseConv2D(kernel_size=(3, 3), padding='same', depth_multiplier=1)(econv)
     a2 = DepthwiseConv2D(kernel_size=(3, 3), strides= 2,padding='same', depth_multiplier=1)(econv)
     a1 = BatchNormalization()(a1)
     a2 = BatchNormalization()(a2)

     b1 = Conv2D(uconv.shape[-1], kernel_size=(3, 3), padding='same')(uconv)
     b2 = DepthwiseConv2D(kernel_size=(3, 3), padding='same', depth_multiplier=1)(uconv)
     b1 = BatchNormalization()(b1)
     b2 = BatchNormalization()(b2)

     a1 = Conv2D(a1.shape[-1], kernel_size=(1,1), padding='same', activation=None)(a1)
     a2 = AveragePooling2D(pool_size=(2, 2), strides=2)(a2)
     
     b1 = UpSampling2D(size=(2, 2), interpolation='nearest')(b1)
     b2 = AveragePooling2D(pool_size=(2, 2), strides=2)(b2)
     b2 = Activation('sigmoid')(b2)

     c1 = multiply([a1, b1])
     c2 = multiply([a2, b2])
     c2 = UpSampling2D(size=(4, 4), interpolation='nearest')(c2)
     sum = add([c1, c2])

     out = Conv2D(sum.shape[-1], kernel_size=(3, 3), padding='same')(sum)
     return BatchNormalization()(out)

In [ ]:
def FPA(x):
  # Global pooling branch
  gp1 = AveragePooling2D(pool_size = x.shape[1])(x)
  gp2 = Conv2D(x.shape[-1],1,padding = 'valid', kernel_initializer='he_normal')(gp1)
  gp3 =  Conv2DTranspose(x.shape[-1], x.shape[1], kernel_initializer='he_normal')(gp2)

  # direct branch
  db = Conv2D(x.shape[-1], 1, padding = 'valid', kernel_initializer='he_normal')(x)

  # U-shape branch
  max_pool_1 = MaxPooling2D(strides = 2)(x)
  
  conv7_1 = Conv2D(x.shape[-1], 7, padding = 'same', kernel_initializer='he_normal')(max_pool_1)
  conv7_1 = BatchNormalization()(conv7_1)
  conv7_1 = Activation('relu')(conv7_1)
  
  conv7_2 = Conv2D(x.shape[-1], 7, padding = 'same', kernel_initializer='he_normal')(conv7_1)
  conv7_2 = BatchNormalization()(conv7_2)
  conv7_2 = Activation('relu')(conv7_2)
  
  max_pool_2 = MaxPooling2D(strides = 2)(conv7_1)
  
  conv5_1 = Conv2D(x.shape[-1], 5, padding = 'same', kernel_initializer='he_normal')(max_pool_2)
  conv5_1 = BatchNormalization()(conv5_1)
  conv5_1 = Activation('relu')(conv5_1)
  
  conv5_2 = Conv2D(x.shape[-1], 5, padding = 'same', kernel_initializer='he_normal')(conv5_1)
  conv5_2 = BatchNormalization()(conv5_2)
  conv5_2 = Activation('relu')(conv5_2)
  
  max_pool_3 = MaxPooling2D(strides = 2)(conv5_1)
  
  conv3_1 = Conv2D(x.shape[-1], 3, padding = 'same', kernel_initializer='he_normal')(max_pool_3)
  conv3_1 = BatchNormalization()(conv3_1)
  conv3_1 = Activation('relu')(conv3_1)
  
  conv3_2 = Conv2D(x.shape[-1], 3, padding = 'same', kernel_initializer='he_normal')(conv3_1)
  conv3_2 = BatchNormalization()(conv3_2)
  conv3_2 = Activation('relu')(conv3_2)
  
  upsampled_8 = Conv2DTranspose(x.shape[-1], 2, strides = (2,2), kernel_initializer='he_normal')(conv3_2)
  
  added_1 = Add()([upsampled_8, conv5_2])
  
  upsampled_16 = Conv2DTranspose(x.shape[-1], 2, strides = (2,2), kernel_initializer='he_normal')(added_1)
  
  added_2 = Add()([upsampled_16, conv7_2])
  
  upsampled_32 = Conv2DTranspose(x.shape[-1], 2, strides = (2,2), kernel_initializer='he_normal')(added_2)

  multiplied = Multiply()([db, upsampled_32])
  return Add()([multiplied, gp3])



In [ ]:
'''
from tensorflow.keras.utils import get_file
import tensorflow as tf
edit: keras_utils to tf.keras.utils 
'''

input_shape = (256,256,3)
dropout_rate = 0.5

backbone = EfficientNetB4(weights='imagenet', include_top=False, input_shape=input_shape)
input = backbone.input
start_neurons = 16

conv4 = backbone.layers[342].output
conv4 = BatchNormalization()(conv4)
conv4 = ReLU()(conv4)
# conv4 = FPA(conv4)
pool4 = MaxPooling2D((2, 2))(conv4) #272
# pool4 = _ASPP(pool4,272) #start_neurons * 32
pool4 = Dropout(dropout_rate)(pool4)

# Middle
convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
convm = BatchNormalization()(convm)
convm = ReLU()(convm)

uconv4 = Conv2D(272, kernel_size=(1,1),activation=None, padding='same')(convm)
econv4 = backbone.layers[342].output
bga1 = BGA(uconv4, econv4)

uconv3 = Conv2D(112, kernel_size=(1,1), activation=None, padding='same' )(bga1)
econv3 = backbone.layers[154].output
bga2 = BGA(uconv3, econv3)

uconv2 = Conv2D(336,kernel_size=(1,1), activation=None, padding='same')(bga2)
econv2 =  backbone.layers[143].output
bga3 = BGA(uconv2, econv2)

uconv1 = Conv2D(192, kernel_size=(1,1), activation=None, padding='same')(bga3)
econv1 = backbone.layers[85].output
bga4 = BGA(uconv1, econv1)

uconv1 = Dropout(0.1)(bga4)
uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
uconv1 = BatchNormalization()(uconv1)
uconv1 = ReLU()(uconv1)
    
uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
uconv0 = Dropout(0.1)(uconv0)
uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
uconv0 = BatchNormalization()(uconv0)
uconv0 = ReLU()(uconv0) 

uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv0)  
uconv0 = Dropout(dropout_rate/2)(uconv0)
output_layer = Conv2D(1, (1,1), padding="same", activation=None)(uconv0)  
# output_layer = FPA(output_layer)
output_layer = Activation('sigmoid')(output_layer)

model = Model(input, output_layer)
model.summary()

## Without ABGA

In [ ]:
# '''
# from tensorflow.keras.utils import get_file
# import tensorflow as tf
# edit: keras_utils to tf.keras.utils 
# '''

# input_shape = (256,256,3)
# dropout_rate = 0.5

# backbone = EfficientNetB4(weights='imagenet', include_top=False, input_shape=input_shape)
# input = backbone.input
# start_neurons = 16

# conv4 = backbone.layers[342].output
# conv4 = BatchNormalization()(conv4)
# conv4 = ReLU()(conv4)
# # conv4 = FPA(conv4)
# pool4 = MaxPooling2D((2, 2))(conv4) #272
# # pool4 = _ASPP(pool4,272) #start_neurons * 32
# pool4 = Dropout(dropout_rate)(pool4)

# # Middle
# convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
# convm = BatchNormalization()(convm)
# convm = ReLU()(convm)

# uconv4 = Conv2DTranspose(272, kernel_size=(3,3),strides=(2,2),activation=None, padding='same')(convm)
# econv4 = backbone.layers[342].output
# uconv4 = concatenate([uconv4, econv4])
# uconv4 = Dropout(dropout_rate)(uconv4)

# uconv3 = Conv2DTranspose(112, kernel_size=(3,3),strides=(2,2), activation=None, padding='same' )(uconv4)
# econv3 = backbone.layers[154].output
# uconv3 = concatenate([uconv3, econv3])
# uconv3 = Dropout(dropout_rate)(uconv3)

# uconv2 = Conv2DTranspose(336,kernel_size=(3,3),strides=(2,2), activation=None, padding='same')(uconv3)
# econv2 =  backbone.layers[143].output
# uconv2 = concatenate([uconv2, econv2])
# uconv2 = Dropout(dropout_rate)(uconv2)

# uconv1 = Conv2DTranspose(192, kernel_size=(3,3),strides=(2,2), activation=None, padding='same')(uconv2)
# econv1 = backbone.layers[85].output
# uconv1 = concatenate([uconv1, econv1])


# uconv1 = Dropout(0.1)(uconv1)
# uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
# uconv1 = BatchNormalization()(uconv1)
# uconv1 = ReLU()(uconv1)
    
# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
# uconv0 = Dropout(0.1)(uconv0)
# uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
# uconv0 = BatchNormalization()(uconv0)
# uconv0 = ReLU()(uconv0) 

# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv0)  
# uconv0 = Dropout(dropout_rate/2)(uconv0)
# output_layer = Conv2D(1, (1,1), padding="same", activation=None)(uconv0)  
# # output_layer = FPA(output_layer)
# output_layer = Activation('sigmoid')(output_layer)

# model = Model(input, output_layer)
# model.summary()

# EffNetB0 version

In [ ]:
# '''
# from tensorflow.keras.utils import get_file
# import tensorflow as tf
# edit: keras_utils to tf.keras.utils 
# '''

# input_shape = (256,256,3)
# dropout_rate = 0.5
# start_neurons = 16

# backbone = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
# input = backbone.input

# conv4 = backbone.layers[214].output
# conv4 = BatchNormalization()(conv4)
# conv4 = ReLU()(conv4)
# pool4 = MaxPooling2D((2, 2))(conv4) 
# #pool4 = _ASPP(pool4,272) #start_neurons * 32
# pool4 = Dropout(dropout_rate)(pool4)

# # Middle
# convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
# convm = BatchNormalization()(convm)
# convm = ReLU()(convm)

# uconv4 = Conv2D(1280, kernel_size=(1,1),activation=None, padding='same')(convm)
# econv4 = backbone.layers[228].output
# bga1 = BGA(uconv4, econv4)

# uconv3 = Conv2D(672, kernel_size=(1,1), activation=None, padding='same' )(bga1)
# econv3 = backbone.layers[157].output
# bga2 = BGA(uconv3, econv3)

# uconv2 = Conv2D(240,kernel_size=(1,1), activation=None, padding='same')(bga2)
# econv2 =  backbone.layers[71].output
# bga3 = BGA(uconv2, econv2)

# uconv1 = Conv2D(144, kernel_size=(1,1), activation=None, padding='same')(bga3)
# econv1 = backbone.layers[43].output
# bga4 = BGA(uconv1, econv1)

# uconv1 = Dropout(0.1)(bga4)
# uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
# uconv1 = BatchNormalization()(uconv1)
# uconv1 = ReLU()(uconv1)
    
# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
# uconv0 = Dropout(0.1)(uconv0)
# uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
# uconv0 = BatchNormalization()(uconv0)
# uconv0 = ReLU()(uconv0) 

# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv0)  
# uconv0 = Dropout(dropout_rate/2)(uconv0)
# output_layer = Conv2D(1, (1,1), padding="same", activation=None)(uconv0)  
# #output_layer = refinement_module(output_layer, 1, 64)
# output_layer = Activation('sigmoid')(output_layer)
# model = Model(input, output_layer)
# model.summary()
# #backbone.summary()

# EffNetB1 version

In [ ]:
# '''
# from tensorflow.keras.utils import get_file
# import tensorflow as tf
# edit: keras_utils to tf.keras.utils 
# '''

# input_shape = (256,256,3)
# dropout_rate = 0.5
# start_neurons = 16

# backbone = EfficientNetB1(weights='imagenet', include_top=False, input_shape=input_shape)
# input = backbone.input

# conv4 = backbone.layers[330].output
# conv4 = BatchNormalization()(conv4)
# conv4 = ReLU()(conv4)
# pool4 = MaxPooling2D((2, 2))(conv4) 
# #pool4 = _ASPP(pool4,272) #start_neurons * 32
# pool4 = Dropout(dropout_rate)(pool4)

# # Middle
# convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
# convm = BatchNormalization()(convm)
# convm = ReLU()(convm)

# uconv4 = Conv2D(1280, kernel_size=(1,1),activation=None, padding='same')(convm)
# econv4 = backbone.layers[330].output
# bga1 = BGA(uconv4, econv4)

# uconv3 = Conv2D(672, kernel_size=(1,1), activation=None, padding='same' )(bga1)
# econv3 = backbone.layers[229].output
# bga2 = BGA(uconv3, econv3)

# uconv2 = Conv2D(240,kernel_size=(1,1), activation=None, padding='same')(bga2)
# econv2 =  backbone.layers[113].output
# bga3 = BGA(uconv2, econv2)

# uconv1 = Conv2D(144, kernel_size=(1,1), activation=None, padding='same')(bga3)
# econv1 = backbone.layers[43].output
# bga4 = BGA(uconv1, econv1)

# uconv1 = Dropout(0.1)(bga4)
# uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
# uconv1 = BatchNormalization()(uconv1)
# uconv1 = ReLU()(uconv1)
    
# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
# uconv0 = Dropout(0.1)(uconv0)
# uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
# uconv0 = BatchNormalization()(uconv0)
# uconv0 = ReLU()(uconv0) 

# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv0)  
# uconv0 = Dropout(dropout_rate/2)(uconv0)
# output_layer = Conv2D(1, (1,1), padding="same", activation=None)(uconv0)  
# # output_layer = refinement_module(output_layer, 1, 64)
# output_layer = Activation('sigmoid')(output_layer)
# model = Model(input, output_layer)
# model.summary()

# EffNetB2 version

In [ ]:
# '''
# from tensorflow.keras.utils import get_file
# import tensorflow as tf
# edit: keras_utils to tf.keras.utils 
# '''

# input_shape = (256,256,3)
# dropout_rate = 0.5
# start_neurons = 16

# backbone = EfficientNetB2(weights='imagenet', include_top=False, input_shape=input_shape)
# input = backbone.input

# conv4 = backbone.layers[330].output
# conv4 = BatchNormalization()(conv4)
# conv4 = ReLU()(conv4)
# pool4 = MaxPooling2D((2, 2))(conv4) 
# #pool4 = _ASPP(pool4,272) #start_neurons * 32
# pool4 = Dropout(dropout_rate)(pool4)

# # Middle
# convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
# convm = BatchNormalization()(convm)
# convm = ReLU()(convm)

# uconv4 = Conv2D(1408, kernel_size=(1,1),activation=None, padding='same')(convm)
# econv4 = backbone.layers[330].output
# bga1 = BGA(uconv4, econv4)

# uconv3 = Conv2D(720, kernel_size=(1,1), activation=None, padding='same' )(bga1)
# econv3 = backbone.layers[229].output
# bga2 = BGA(uconv3, econv3)

# uconv2 = Conv2D(288,kernel_size=(1,1), activation=None, padding='same')(bga2)
# econv2 =  backbone.layers[113].output
# bga3 = BGA(uconv2, econv2)

# uconv1 = Conv2D(144, kernel_size=(1,1), activation=None, padding='same')(bga3)
# econv1 = backbone.layers[70].output
# bga4 = BGA(uconv1, econv1)

# uconv1 = Dropout(0.1)(bga4)
# uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
# uconv1 = BatchNormalization()(uconv1)
# uconv1 = ReLU()(uconv1)
    
# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
# uconv0 = Dropout(0.1)(uconv0)
# uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
# uconv0 = BatchNormalization()(uconv0)
# uconv0 = ReLU()(uconv0) 

# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv0)  
# uconv0 = Dropout(dropout_rate/2)(uconv0)
# output_layer = Conv2D(1, (1,1), padding="same", activation=None)(uconv0)  
# # output_layer = refinement_module(output_layer, 1, 64)
# output_layer = Activation('sigmoid')(output_layer)
# model = Model(input, output_layer)
# model.summary()


#EffNetB3 version

In [ ]:
# '''
# from tensorflow.keras.utils import get_file
# import tensorflow as tf
# edit: keras_utils to tf.keras.utils 
# '''

# input_shape = (256,256,3)
# dropout_rate = 0.5
# start_neurons = 16

# backbone = EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)
# input = backbone.input

# conv4 = backbone.layers[375].output
# conv4 = BatchNormalization()(conv4)
# conv4 = ReLU()(conv4)
# pool4 = MaxPooling2D((2, 2))(conv4) 
# #pool4 = _ASPP(pool4,272) #start_neurons * 32
# pool4 = Dropout(dropout_rate)(pool4)

# # Middle
# convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
# convm = BatchNormalization()(convm)
# convm = ReLU()(convm)

# uconv4 = Conv2D(1536, kernel_size=(1,1),activation=None, padding='same')(convm)
# econv4 = backbone.layers[375].output
# bga1 = BGA(uconv4, econv4)

# uconv3 = Conv2D(816, kernel_size=(1,1), activation=None, padding='same' )(bga1)
# econv3 = backbone.layers[259].output
# bga2 = BGA(uconv3, econv3)

# uconv2 = Conv2D(288,kernel_size=(1,1), activation=None, padding='same')(bga2)
# econv2 =  backbone.layers[113].output
# bga3 = BGA(uconv2, econv2)

# uconv1 = Conv2D(192, kernel_size=(1,1), activation=None, padding='same')(bga3)
# econv1 = backbone.layers[70].output
# bga4 = BGA(uconv1, econv1)

# uconv1 = Dropout(0.1)(bga4)
# uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
# uconv1 = BatchNormalization()(uconv1)
# uconv1 = ReLU()(uconv1)
    
# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
# uconv0 = Dropout(0.1)(uconv0)
# uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
# uconv0 = BatchNormalization()(uconv0)
# uconv0 = ReLU()(uconv0) 

# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv0)  
# uconv0 = Dropout(dropout_rate/2)(uconv0)
# output_layer = Conv2D(1, (1,1), padding="same", activation=None)(uconv0)  
# # output_layer = refinement_module(output_layer, 1, 64)
# output_layer = Activation('sigmoid')(output_layer)
# model = Model(input, output_layer)
# model.summary()

#EffNet B4 version

In [ ]:

# '''
# from tensorflow.keras.utils import get_file
# import tensorflow as tf
# edit: keras_utils to tf.keras.utils 
# '''

# input_shape = (256,256,3)
# dropout_rate = 0.5
# start_neurons = 16

# backbone = EfficientNetB4(weights='imagenet', include_top=False, input_shape=input_shape)
# input = backbone.input

# conv4 = backbone.layers[465].output
# conv4 = BatchNormalization()(conv4)
# conv4 = ReLU()(conv4)
# pool4 = MaxPooling2D((2, 2))(conv4) 
# #pool4 = _ASPP(pool4,272) #start_neurons * 32
# pool4 = Dropout(dropout_rate)(pool4)

# # Middle
# convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
# convm = BatchNormalization()(convm)
# convm = ReLU()(convm)

# # convm = FPA(convm)

# uconv4 = Conv2D(1792, kernel_size=(1,1),activation=None, padding='same')(convm)
# econv4 = backbone.layers[465].output
# bga1 = BGA(uconv4, econv4)

# uconv3 = Conv2D(960, kernel_size=(1,1), activation=None, padding='same' )(bga1)
# econv3 = backbone.layers[319].output
# bga2 = BGA(uconv3, econv3)

# uconv2 = Conv2D(336,kernel_size=(1,1), activation=None, padding='same')(bga2)
# econv2 =  backbone.layers[143].output
# bga3 = BGA(uconv2, econv2)

# uconv1 = Conv2D(192, kernel_size=(1,1), activation=None, padding='same')(bga3)
# econv1 = backbone.layers[85].output
# bga4 = BGA(uconv1, econv1)

# uconv1 = Dropout(0.1)(bga4)
# uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
# uconv1 = BatchNormalization()(uconv1)
# uconv1 = ReLU()(uconv1)
    
# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
# uconv0 = Dropout(0.1)(uconv0)
# uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
# uconv0 = BatchNormalization()(uconv0)
# uconv0 = ReLU()(uconv0) 

# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv0)  
# uconv0 = Dropout(dropout_rate/2)(uconv0)
# output_layer = Conv2D(1, (1,1), padding="same", activation=None)(uconv0)  
# # output_layer = refinement_module(output_layer, 1, 64)
# output_layer = Activation('sigmoid')(output_layer)
# model = Model(input, output_layer)
# model.summary()

## Salient Attention Unet (based SOD)

In [ ]:
'''input_shape = (256,256,3)
input = Input(shape=input_shape)

conv1 = UNetBlock(input, 64)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
input2 = Input(shape=input_shape) #not having saliency map
    
dwns1 = MaxPooling2D(2,2)(input2)
attn1 = SalientAttentionBlock(conv1, dwns1, pool1, 64)

conv2 = UNetBlock(attn1, 64)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

dwns2 = MaxPooling2D(4,4)(input2)
attn2 = SalientAttentionBlock(conv2, dwns2, pool2, 64)

conv3 = UNetBlock(attn2, 128)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

dwns3 = MaxPooling2D(8,8)(input2)
attn3 = SalientAttentionBlock(conv3, dwns3, pool3, 128)

conv4 = UNetBlock(attn3, 128)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

dwns4 = MaxPooling2D(16,16)(input2)
attn4 = SalientAttentionBlock(conv4, dwns4, pool4, 128)

conv5 = UNetBlock(attn4, 256)
pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)

dwns5 = MaxPooling2D(32,32)(input2)
attn5 = SalientAttentionBlock(conv5, dwns5, pool5, 256)

conv6 = UNetBlock(attn5, 256)
pool6 = MaxPooling2D(pool_size=(2, 2))(conv6)

dwns6 = MaxPooling2D(64,64)(input2)
attn6 = SalientAttentionBlock(conv6, dwns6, pool6, 256)

conv7 = UNetBlock(attn6, 512)

up8 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv7), attn5], axis=3)
conv8 = UNetBlock(up8, 256)

up9 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv8), attn4], axis=3)
conv9 = UNetBlock(up9, 256)

up10 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv9), attn3], axis=3)
conv10 = UNetBlock(up10, 128)

up11 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv10), attn2], axis=3)
conv11 = UNetBlock(up11, 128)

up12 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv11), attn1], axis=3)
conv12 = UNetBlock(up12, 64)

up13 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv12), conv1], axis=3)
conv13 = UNetBlock(up13, 64)

conv14 = Conv2D(1, (1, 1), activation='sigmoid')(conv13)
model = Model(inputs = [input, input2], outputs = conv14)
model.summary()'''

## Seg-UNet

In [ ]:
'''input_shape = (256,256,3)
encoder = encoderSegnet(input_s = input_shape)
skip_connect=[encoder.get_layer(i).output for i in list_skip]
num_filters = [512,256, 128, 64]

o = encoder.output
o = _ASPP(o,128)
  
for i, f in enumerate(num_filters):
  o = decoder_block(f,skip=skip_connect[i])(o)
  
o = Conv2D(1,(3, 3), padding='same', kernel_initializer='he_normal')(o)
# yn = Activation('softmax')(o[...,:-1])
# bn = o[...,-1:]
# output = Concatenate()([yn,bn])
#if out_channels > 1 : 
  #output = Activation('softmax', name = 'softmax')(o)
#else :
output = Activation('sigmoid', name = 'sigmoid')(o)
model4 = Model(encoder.input,output)
model4.summary()'''

## ResUNet

In [ ]:
# f = [16, 32, 64, 128, 256]
# input_shape = (256,256,3)
# inputs = Input(shape=input_shape, dtype='float', name='data')

# ## ENCODER 
# e0 = inputs
# e1 = stem(e0, f[0])
# e2 = _residual_block(e1, f[1], strides=2)
# e3 = _residual_block(e2, f[2], strides=2)
# e4 = _residual_block(e3, f[3], strides=2)
# e5 = _residual_block(e4, f[4], strides=2)

# # BRIDGE
# b0 = _conv_block(e5, f[4], strides=1)
# b1 = _conv_block(b0, f[4], strides=1)

# # DECODER 
# u1 = upsample_concat_block(b1, e4)
# d1 = _residual_block(u1, f[4])

# u2 = upsample_concat_block(d1, e3)
# d2 = _residual_block(u2, f[3])

# u3 = upsample_concat_block(d2, e2)
# d3 = _residual_block(u3, f[2])

# u4 = upsample_concat_block(d3, e1)
# d4 = _residual_block(u4, f[1])

# outputs = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(d4)
# model = Model(inputs, outputs)

# model.summary()

## EfficientUNet B0-B7

In [ ]:
#%run /content/drive/MyDrive/brain_mri_segmentation/EfficientUNet.ipynb #0.9100

In [ ]:
#model2 = get_efficient_unet_b2((256,256,3), pretrained=True, block_type='transpose', concat_input=True,out_channels=1)
#model2.summary()

## DeepLab V3

In [ ]:
'''input_shape = (256,256,3)
input = Input(shape=input_shape)
resnet50 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, input_tensor=input)
x = resnet50.get_layer("conv4_block6_2_relu").output
x = DilatedSpatialPyramidPooling(x)

input_a = layers.UpSampling2D(size=(256 // 4 // x.shape[1], 256 // 4 // x.shape[2]),interpolation="bilinear",)(x)
input_b = resnet50.get_layer("conv2_block3_2_relu").output
input_b = _convolution_block(input_b, num_filters=48, kernel_size=1)

x = layers.Concatenate(axis=-1)([input_a, input_b])
x = _convolution_block(x)
x = _convolution_block(x)
x = layers.UpSampling2D(size=(256 // x.shape[1], 256 // x.shape[2]),interpolation="bilinear",)(x)
model_output = layers.Conv2D(1, kernel_size=(1, 1), padding="same")(x)
model = Model(inputs=input, outputs=model_output)
model = Deeplabv3(input_shape=(256,256,3),backbone="mobilenetv2", classes=1)
model.summary()'''

## Swin U-Net

In [ ]:
# filter_num_begin = 32     # number of channels in the first downsampling block; it is also the number of embedded dimensions
# depth = 4                  # the depth of SwinUNET; depth=4 means three down/upsampling levels and a bottom level 
# stack_num_down = 2         # number of Swin Transformers per downsampling level
# stack_num_up = 2           # number of Swin Transformers per upsampling level
# patch_size = (4, 4)        # Extract 2-by-2 patches from the input image. Height and width of the patch must be equal.
# num_heads = [4, 8, 8, 8]   # number of attention heads per down/upsampling level
# window_size = [4, 2, 2, 2] # the size of attention window per down/upsampling level
# num_mlp = 512              # number of MLP nodes within the Transformer
# shift_window=True          # Apply window shifting, i.e., Swin-MSA

# model = models.swin_unet_2d(input_size=(256,256,3), filter_num_begin=filter_num_begin, n_labels=1, depth=depth, stack_num_down=stack_num_down, stack_num_up=stack_num_up, 
#                       patch_size=patch_size, num_heads=num_heads, window_size=window_size, num_mlp=num_mlp, output_activation='Sigmoid', shift_window=True, name='swin_unet')
# model.summary()

## TransU-Net

In [ ]:
'''model = models.transunet_2d(input_size=(256,256,3), filter_num=[32, 64, 128, 256], n_labels=1, stack_num_down=2, stack_num_up=2,
                 embed_dim=768, num_mlp = 3072, num_heads=12, num_transformer=12,
                 activation='ReLU', mlp_activation='GELU', output_activation='Softmax', batch_norm=False, pool=True, unpool=True, 
                 backbone=None, weights='imagenet', freeze_backbone=True, freeze_batch_norm=True, name='transunet')
model.summary()'''

## U-Net++

In [ ]:
# start_neurons = 8

# input_shape = (256,256,3)
# data = Input(shape=input_shape, dtype='float', name='data')

# mvn0 = BatchNormalization()(data)
# conv0 = Conv2D(8, 3, padding = 'same')(mvn0)
# conv0 = BatchNormalization()(conv0)
# conv0 = Activation('relu')(conv0)
# conv0 = Conv2D(8, 3,  padding = 'same')(conv0)
# conv0 = BatchNormalization()(conv0)
# conv0 = Activation('relu')(conv0)
# pool0 = MaxPooling2D(pool_size=(2, 2))(conv0)

# conv1 = Conv2D(16, 3,  padding = 'same')(pool0)
# conv1 = BatchNormalization()(conv1)
# conv1 = Activation('relu')(conv1)
# conv1 = Conv2D(16, 3,  padding = 'same')(conv1)
# conv1 = BatchNormalization()(conv1)
# conv1 = Activation('relu')(conv1)
# pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

# conv2 = Conv2D(32, 3,  padding = 'same')(pool1)
# conv2 = BatchNormalization()(conv2)
# conv2 = Activation('relu')(conv2)
# conv2 = Conv2D(32, 3,  padding = 'same')(conv2)
# conv2 = BatchNormalization()(conv2)
# conv2 = Activation('relu')(conv2)
# pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
# conv3 = Conv2D(64, 3,  padding = 'same')(pool2)
# conv3 = BatchNormalization()(conv3)
# conv3 = Activation('relu')(conv3)
# conv3 = Conv2D(64, 3,  padding = 'same')(conv3)
# conv3 = BatchNormalization()(conv3)
# conv3 = Activation('relu')(conv3)
# pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

# conv4 = Conv2D(128, 3,  padding = 'same')(pool3)
# conv4 = BatchNormalization()(conv4)
# conv4 = Activation('relu')(conv4)
# conv4 = Conv2D(128, 3,  padding = 'same')(conv4)
# conv4 = BatchNormalization()(conv4)
# conv4 = Activation('relu')(conv4)
# drop4 = Dropout(0.5)(conv4)
# pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
# # Middle
# convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same",name='conv_middle')(pool4)
# convm = ReLU()(convm)
    
# deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
# deconv4_up1 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(deconv4)
# deconv4_up2 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(deconv4_up1)
# deconv4_up3 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(deconv4_up2)
# uconv4 = concatenate([deconv4, conv4])
# uconv4 = Dropout(0.5)(uconv4) 
    
# uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
# uconv4 = ReLU()(uconv4)  #conv1_2
    
# deconv3 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
# deconv3_up1 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(deconv3)
# deconv3_up2 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(deconv3_up1)
# uconv3 = concatenate([deconv3,deconv4_up1, conv3])    
# uconv3 = Dropout(0.5)(uconv3)
    
# uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
# uconv3 = ReLU()(uconv3)
# deconv2 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
# deconv2_up1 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(deconv2)
# uconv2 = concatenate([deconv2,deconv3_up1,deconv4_up2, conv2])
        
# uconv2 = Dropout(0.2)(uconv2)
# uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
# uconv2 = ReLU()(uconv2)
    
# deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
# uconv1 = concatenate([deconv1,deconv2_up1,deconv3_up2,deconv4_up3, conv1])
# uconv1 = Dropout(0.2)(uconv1)
# uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
# uconv1 = ReLU()(uconv1)
    
# uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
# uconv0 = Dropout(0.2)(uconv0)
# uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
# uconv0 = ReLU()(uconv0)
    
# uconv0 = Dropout(0.2)(uconv0)
# output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    
    
# model = Model(data, output_layer)
# model.summary()

## PraNet

In [ ]:
'''input_shape = (256,256,3)
input = Input(shape=input_shape)

fe_backbone = FE_backbone(model_architecture='resnet50', inshape=input_shape, is_trainable=True)
backbone_feature_extractor = fe_backbone.get_fe_backbone()
features = backbone_feature_extractor(input)

# RFB
feat2_rfb = RFB(filters=32, name="rfb_2")(features[1])  # => level_2(batch,h/8,w/8,32)
feat3_rfb = RFB(filters=32, name="rfb_3")(features[2])  # => level_3(batch,h/16,w/16,32)
feat4_rfb = RFB(filters=32, name="rfb_4")(features[3])  # => level_4(batch,h/32,w/32,32)

# Partial decoder 
sg = PartialDecoder(filters=32, name="partial_decoder")(feat4_rfb, feat3_rfb, feat2_rfb) # => (batch,h/8,w/8,1) Global saliency map
lateral_out_sg = preprocessing.Resizing(256, 256, name='salient_out_5')(sg) # resize (batch,h/8,w/8,1) => (batch,h,w,1) #out 5

# reverse attention branch 4 
resized_sg = preprocessing.Resizing(256//32, 256//32, name="resize4")(sg) # resize (batch, h/8,w/8,1) => (batch, h/32,w/32,1)
s4 = ReverseAttention(filters=256, kernel_size=(5, 5),branch='gsmap', name="reverse_attention_br4")(features[3],resized_sg)
lateral_out_s4 = preprocessing.Resizing(256, 256, name="salient_out_4")(s4) # resize (batch,h/32,w/32,1) => (batch,h,w,1) #out 4

# reverse attention branch 3
resized_s4 = preprocessing.Resizing(256//16, 256//16, name="resize3")(s4) # resize (batch, h/32,w/32,1) => (batch, h/16,w/16,1)
s3 = ReverseAttention(filters=64, name="reverse_attention_br3")(features[2],resized_s4)
lateral_out_s3 = preprocessing.Resizing(256, 256, name="salient_out_3")(s3) # resize (batch,h/16,w/16,1) => (batch,h,w,1) #out 3

# reverse attention branch 2
resized_s3 = preprocessing.Resizing(256//8, 256//8, name="resize2")(s3)# resize (batch, h/16,w/16,1) => (batch, h/8,w/8,1)
s2 = ReverseAttention(filters=64, name="reverse_attention_br2")(features[1],resized_s3)
lateral_out_s2 = preprocessing.Resizing(256, 256, name="final_salient_out_2")(s2)# resize (batch,h/8,w/8,1) => (batch,h,w,1) #out 2
lateral_out_s2 = Activation('sigmoid')(lateral_out_s2)

model = Model(inputs = input, outputs = lateral_out_s2)
model.summary()'''
#0.8916

## Double U-Net

In [ ]:
# input_shape = (256,256,3)
# inputs = Input(input_shape)
# x, skip_1 = encoder1(inputs)
# x = _ASPP_(x, 64)
# x = decoder1(x, skip_1)
# outputs1 = output_block(x)

# x = inputs * outputs1

# x, skip_2 = encoder2(x)
# x = _ASPP_(x, 64)
# x = decoder2(x, skip_1, skip_2)
# outputs2 = output_block(x)
# outputs = Concatenate()([outputs1, outputs2])

# model3 = Model(inputs, outputs)
# model3.summary()

## Metric

In [ ]:
def dice_coef(y_true, y_pred, smooth=1e-10):
    '''Average dice coefficient per batch.'''
    #y_true = tf.one_hot(tf.squeeze(tf.cast(y_true, tf.uint8), axis=-1), depth=2)
    #y_true = tf.cast(y_true, tf.float32)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    summation = K.sum(y_true + y_pred)
    return (2.0 * intersection + smooth) / (summation + smooth)

def jaccard_coef(y_true, y_pred, smooth=1e-10):
    '''Average jaccard coefficient per batch.'''
    #y_true = tf.one_hot(tf.squeeze(tf.cast(y_true, tf.uint8), axis=-1), depth=2)
    #y_true = tf.cast(y_true, tf.float32)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true + y_pred) - intersection
    return (intersection + smooth) / (union + smooth)

def TP(target, output, smooth=1e-10):
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)
    true_pos = K.sum(target_positive * output_positive)

    return true_pos

def TN(target, output, smooth=1e-10):
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)
    true_neg = K.sum((1-target_positive) * (1-output_positive))

    return true_neg

def FP(target, output, smooth=1e-10):
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)
    false_pos = K.sum((1-target_positive) * output_positive)

    return false_pos

def FN(target, output, smooth=1e-10):
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)
    false_neg = K.sum(target_positive * (1-output_positive))

    return false_neg

In [ ]:
def dice_coef_loss(y_true, y_pred, smooth=1e-10):
    y_true = tf.cast(y_true, tf.float32)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    summation = K.sum(y_true + y_pred)
    return 1-(2.0 * intersection + smooth) / (summation + smooth)

def IoU_loss(y_true, y_pred, smooth=1e-10):
    '''Average jaccard coefficient per batch.'''
    #y_true = tf.one_hot(tf.squeeze(tf.cast(y_true, tf.uint8), axis=-1), depth=2)
    y_true = tf.cast(y_true, tf.float32)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true + y_pred) - intersection
    return 1-(intersection + smooth) / (union + smooth)

def tversky_loss(target, output):
    #target = tf.one_hot(tf.squeeze(tf.cast(target, tf.uint8), axis=-1), depth=2)
    target = tf.cast(target, tf.float32)
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)

    true_pos = K.sum(target_positive * output_positive)
    true_neg = K.sum((1-target_positive) * (1-output_positive))
    false_neg = K.sum(target_positive * (1-output_positive))
    false_pos = K.sum((1-target_positive) * output_positive)
    alpha = 0.7
    loss = 1-(true_pos)/(true_pos + alpha*false_neg + (1-alpha)*false_pos)
    return loss

def focal_tversky(target, output):
    p = tversky_loss(target, output)
    gamma = 0.75
    return K.pow(p, gamma)

def tversky_kahneman(target, output):
    alpha=0.5; beta=0.5; gamma=4/3; smooth=1e-10;
    #target = tf.one_hot(tf.squeeze(tf.cast(target, tf.uint8), axis=-1), depth=2)
    target = tf.cast(target, tf.float32)
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)

    true_pos = K.sum(target_positive * output_positive)
    true_neg = K.sum((1-target_positive) * (1-output_positive))
    false_neg = K.sum(target_positive * (1-output_positive))
    false_pos = K.sum((1-target_positive) * output_positive)
    _true_pos = K.sqrt(true_pos*true_neg)

    p = (0.5*_true_pos + 0.5*true_neg)/(0.5*_true_pos + 0.5*true_neg + alpha*false_pos + beta*false_neg)  
    p_gamma = K.pow(p,gamma) #p^gamma
    _p_gamma = K.pow(1-p, gamma) #(1-p)^gamma
    loss = _p_gamma/K.pow(p_gamma + _p_gamma, 1/gamma)
    return loss

def accuracy_loss(target, output):
    target = tf.cast(target, tf.float32)
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)

    true_pos = K.sum(target_positive * output_positive)
    true_neg = K.sum((1-target_positive) * (1-output_positive))
    false_neg = K.sum(target_positive * (1-output_positive))
    false_pos = K.sum((1-target_positive) * output_positive)

    a = 1-(true_pos + true_neg)/(true_pos + true_neg + false_pos + false_neg)
    return a

def bce_loss(target, output):
    target = tf.cast(target, tf.float32)
    _target = K.flatten(target)
    _output = K.flatten(output)

    loss = binary_crossentropy(_target, _output)
    return loss

def bub_loss(target, output):
    #target = tf.one_hot(tf.squeeze(tf.cast(target, tf.uint8), axis=-1), depth=2)
    target = tf.cast(target, tf.float32)
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)

    true_pos = K.sum(target_positive * output_positive)
    true_neg = K.sum((1-target_positive) * (1-output_positive))
    false_neg = K.sum(target_positive * (1-output_positive))
    false_pos = K.sum((1-target_positive) * output_positive)
    _true_neg = K.sqrt(true_pos*true_neg)

    b = 1-(true_pos + _true_neg)/(true_pos + _true_neg + false_pos + false_neg)
    return b

In [ ]:
def dice_1_loss(target, output):
    target = tf.cast(target, tf.float32)
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)

    true_pos = K.sum(target_positive * output_positive)
    true_neg = K.sum((1-target_positive) * (1-output_positive))
    false_neg = K.sum(target_positive * (1-output_positive))
    false_pos = K.sum((1-target_positive) * output_positive)

    loss = 1 - true_pos / (true_pos + false_neg)
    return loss

def dice_2_loss(target, output):
    target = tf.cast(target, tf.float32)
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)

    true_pos = K.sum(target_positive * output_positive)
    true_neg = K.sum((1-target_positive) * (1-output_positive))
    false_neg = K.sum(target_positive * (1-output_positive))
    false_pos = K.sum((1-target_positive) * output_positive)

    loss = 1-true_pos / (true_pos + false_pos)
    return loss

def swjaccard_loss(target, output):
    target = tf.cast(target, tf.float32)
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)

    true_pos = K.sum(target_positive * output_positive)
    true_neg = K.sum((1-target_positive) * (1-output_positive))
    false_neg = K.sum(target_positive * (1-output_positive))
    false_pos = K.sum((1-target_positive) * output_positive)

    loss = 1 - 3*true_pos / (3*true_pos + false_neg + false_pos)
    return loss

def Kulczynski_1_loss(target, output):
    target = tf.cast(target, tf.float32)
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)

    true_pos = K.sum(target_positive * output_positive)
    true_neg = K.sum((1-target_positive) * (1-output_positive))
    false_neg = K.sum(target_positive * (1-output_positive))
    false_pos = K.sum((1-target_positive) * output_positive)

    loss = 1 - true_pos / (false_pos + false_neg)
    return loss

def Kulczynski_2_loss(target, output):
    target = tf.cast(target, tf.float32)
    target_positive = K.flatten(target)
    output_positive = K.flatten(output)

    true_pos = K.sum(target_positive * output_positive)
    true_neg = K.sum((1-target_positive) * (1-output_positive))
    false_neg = K.sum(target_positive * (1-output_positive))
    false_pos = K.sum((1-target_positive) * output_positive)

    loss = 1 - 0.5*(true_pos/(true_pos + false_neg) + true_pos/(true_pos + false_pos))
    return loss

def driver_kroeber_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - true_pos / (np.sqrt((true_pos + false_neg)*(true_pos + false_pos)))
  return loss

def braun_blanquet(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - true_pos/(np.max((true_pos + false_neg, true_pos + false_pos)))
  return loss

def simpson_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - true_pos/(np.min((true_pos + false_neg, true_pos + false_pos)))

def Sorgenfrei_loss(target, input):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - true_pos*true_pos / ((true_pos + false_neg)*(true_pos + false_pos))
  return loss

def Mountford_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - 2*true_pos / (true_pos*(false_neg + false_pos) + 2*false_pos*false_neg)
  return loss

def fager_mcgowan_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - (true_pos / (np.sqrt((true_pos + false_neg)*(true_pos + false_pos))) - np.max((true_pos + false_neg, true_pos + false_pos)) / 2)
  return loss

def sokal_sneath_1_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - true_pos / (true_pos + 2*(false_neg + false_pos))
  return loss

def mcconaughey_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - (true_pos*true_pos - false_neg*false_pos) / ((true_pos + false_neg)*(true_pos + false_pos))
  return loss

def johnson_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - true_pos*(1/(true_pos + false_neg) + 1/(true_pos + false_pos))
  return loss

def van_der_maarel_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - (2*true_pos - false_neg - false_pos)/(2*true_pos + false_neg + false_pos)
  return loss

def consonni_todeschini_1_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - np.log(1 + true_pos)/np.log(1 + true_pos + false_neg + false_pos)
  return loss

def rushell_rao_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - true_pos / (true_pos + false_neg + false_pos + true_neg)
  return loss

def consonni_todeschini_2_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - np.log(1 + true_pos)/np.log(1 + true_pos + false_neg + false_pos + true_neg)
  return loss

def sokal_michener_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - (true_pos + true_neg) / (true_pos + false_neg + false_pos + true_neg)
  return loss

def rogers_tanimoto_loss(target, output):
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  loss = 1 - (true_pos + true_neg) / (true_pos + 2*false_neg + 2*false_pos + true_neg)
  return loss

## Training

In [ ]:
# compling model and callbacks functions
nadam = tf.keras.optimizers.Nadam(0.005)
model.compile(optimizer = nadam, loss = Kulczynski_2_loss, metrics = [jaccard_coef, dice_coef])
#model1.compile(optimizer = nadam, loss = tversky_kahneman, metrics = [jaccard_coef, dice_coef]) #tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
#model2.compile(optimizer = nadam, loss = tversky_kahneman, metrics = [jaccard_coef, dice_coef])
#model3.compile(optimizer = nadam, loss = tversky_kahneman, metrics = [jaccard_coef, dice_coef])
#model4.compile(optimizer = nadam, loss = tversky_kahneman, metrics = [jaccard_coef, dice_coef])
#model5.compile(optimizer = nadam, loss = tversky_kahneman, metrics = [jaccard_coef, dice_coef])

#callbacks
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min', 
                              verbose=1, 
                              patience=20)
# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath="/content/drive/MyDrive/brain_mri_segmentation/weights/cell/weight.h5", 
                               monitor="val_loss",
                               verbose=1, 
                               save_best_only=True) #filepath="/content/drive/MyDrive/brain_mri_segmentation/weight/weight.h5"

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=8,
                              min_delta=0.0001,
                              min_lr=1e-5,
                              factor=0.5)

In [ ]:
h = model.fit(x_train, y_train, validation_data = [x_test,y_test], batch_size=8, epochs = 200, callbacks = [checkpointer, reduce_lr])

## Drawing for Checking

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

# model1.load_weights('/content/drive/MyDrive/brain_mri_segmentation/weight/fcn_cell_weight.h5')
# model2.load_weights('/content/drive/MyDrive/brain_mri_segmentation/weight/effunet_cell_weight.h5')
# model3.load_weights('/content/drive/MyDrive/brain_mri_segmentation/weight/doubleunet_cell_weight.h5')
# model4.load_weights('/content/drive/MyDrive/brain_mri_segmentation/weight/seg_unet_cell_weight.h5')
# model5.load_weights('/content/drive/MyDrive/brain_mri_segmentation/weight/proposed_cell_weight.h5')
model.load_weights('/content/drive/MyDrive/brain_mri_segmentation/weights/cell/weight.h5')




In [ ]:
fig, axs = plt.subplots(4, 3, figsize = (20,30), dpi=300)
for i in range(4):
  y_pred = model(x_test[i:i+1])
  # plt.subplot(131),plt.imshow(x_test[i]), plt.axis('off')
  # plt.subplot(132),plt.imshow(y_pred[0,...,0],cmap ='gray'), plt.axis('off')
  # plt.subplot(133),plt.imshow(y_test[i,...,0],cmap='gray'), plt.axis('off')
  axs[i, 0].imshow(x_test[i])
  axs[i, 1].imshow(y_pred[0,...,0],cmap ='gray')
  axs[i, 2].imshow(y_test[i,...,0],cmap='gray')
  axs[i, 0].set_axis_off()
  axs[i, 1].set_axis_off()
  axs[i, 2].set_axis_off()
plt.subplots_adjust(wspace=0.1, hspace=0.1)
axs[0,0].set_title('Input Image',x=0.5, y=1.1, fontsize=50)
axs[0,1].set_title('Ground Truth',x=0.5, y=1.1, fontsize=50)
axs[0,2].set_title('Prediction',x=0.5, y=1.1, fontsize=50)
plt.savefig(f'/content/drive/MyDrive/brain_mri_segmentation/weights/fig/data.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
h.history.keys()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize = (20,25), dpi=100)
axs[0].plot(h.history['dice_coef'])
axs[0].plot(h.history['val_dice_coef'])
axs[0].legend(['train', 'validation'], loc='lower right', fontsize = 20)
axs[0].set_title('model accuracy',x=0.5, y=1,fontsize=40)
axs[0].tick_params(axis='both', which='major', labelsize=20)

axs[1].plot(h.history['jaccard_coef'])
axs[1].plot(h.history['val_jaccard_coef'])
axs[1].legend(['train', 'validation'], loc='lower right', fontsize = 20)
axs[1].set_title('model IoU',x=0.5, y=1, fontsize=40)
axs[1].tick_params(axis='both', which='major', labelsize=20)
axs[2].plot(h.history['loss'])
axs[2].plot(h.history['val_loss'])
axs[2].legend(['train', 'validation'], loc='upper right', fontsize = 20)
axs[2].set_title('model loss',x=0.5, y=1, fontsize=40)
axs[2].tick_params(axis='both', which='major', labelsize=20)
plt.savefig(f'/content/drive/MyDrive/brain_mri_segmentation/weights/fig/plot.pdf', bbox_inches='tight', pad_inches=0)
